In [1]:
import requests
import numpy as np
import pandas as pd
from transformers import pipeline
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report


C:\Users\joana\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Load Data

In [2]:
# Load the CSV file into a DataFrame
#df_en = pd.read_csv('../Preprocessing/data_clean/labeled-data/labeled-en_clean_1-1_not101010.csv')

In [3]:
df_en = pd.read_csv('https://raw.githubusercontent.com/svisel22/SS23-BIPM-Analytics-Lab---Group-4-repository/main/Preprocessing/data_clean/labeled-data/labeled-en_clean_con_sen.csv')

In [4]:
df_en.head()

,data,player,language,publishedAt,Label
0,bayerleverkusen took lead minute midfielder p...,exequiel palacios,en,2023-02-23T20:50:50Z,NaN
1,midfielders leandro paredes juventus angel ...,exequiel palacios,en,2023-03-03T16:42:19Z,neutral
2,half goal joshua kimmich canceled penalties pa...,exequiel palacios,en,2023-03-19T18:30:00Z,positiv
3,by reuters bayerleverkusen s palacios scored s...,exequiel palacios,en,2023-03-19T18:42:59Z,NaN
4,bayerleverkusen s palacios scored second half ...,exequiel palacios,en,2023-03-19T19:05:09Z,positiv


# Fix NA

In [5]:
#for DE datasate drop empty lines
df_de = df_en.dropna(subset=['data'])

In [6]:
df_en.head()

,data,player,language,publishedAt,Label
0,bayerleverkusen took lead minute midfielder p...,exequiel palacios,en,2023-02-23T20:50:50Z,NaN
1,midfielders leandro paredes juventus angel ...,exequiel palacios,en,2023-03-03T16:42:19Z,neutral
2,half goal joshua kimmich canceled penalties pa...,exequiel palacios,en,2023-03-19T18:30:00Z,positiv
3,by reuters bayerleverkusen s palacios scored s...,exequiel palacios,en,2023-03-19T18:42:59Z,NaN
4,bayerleverkusen s palacios scored second half ...,exequiel palacios,en,2023-03-19T19:05:09Z,positiv


# Sentiment analysis full text

In [7]:
sentiment_classifier = pipeline('sentiment-analysis', model='bert-base-uncased')


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

In [8]:
def perform_comparison_sentiment_analysis(df):
    # Create empty lists to store the sentiment scores and labels
    sentiment_scores = []
    sentiment_labels = []

    # Iterate over the 'data' column in the DataFrame
    for text in df['data']:
        # Perform sentiment analysis using the Hugging Face pipeline
        result = sentiment_classifier(text)[0]
        sentiment_score = result['score']
        
        # Convert logits to predicted label (positive/negative)
        predicted_label = 1 if sentiment_score > 0.5 else 0
        sentiment_label = "positiv" if predicted_label == 1 else "negativ"

        # Append the sentiment score and label to the respective lists
        sentiment_scores.append(sentiment_score)
        sentiment_labels.append(sentiment_label)

    # Add the sentiment scores and labels as new columns in the DataFrame
    df['sentiment_score'] = sentiment_scores
    df['sentiment_label'] = sentiment_labels

    return df


# Perform sentiment analysis on the DataFrame
df = perform_comparison_sentiment_analysis(df_en)

# Print the updated DataFrame
df.head()


,data,player,language,publishedAt,Label,sentiment_score,sentiment_label
0,bayerleverkusen took lead minute midfielder p...,exequiel palacios,en,2023-02-23T20:50:50Z,NaN,0.632718,positiv
1,midfielders leandro paredes juventus angel ...,exequiel palacios,en,2023-03-03T16:42:19Z,neutral,0.650659,positiv
2,half goal joshua kimmich canceled penalties pa...,exequiel palacios,en,2023-03-19T18:30:00Z,positiv,0.612918,positiv
3,by reuters bayerleverkusen s palacios scored s...,exequiel palacios,en,2023-03-19T18:42:59Z,NaN,0.612275,positiv
4,bayerleverkusen s palacios scored second half ...,exequiel palacios,en,2023-03-19T19:05:09Z,positiv,0.632688,positiv


In [9]:
'''three labels:
def perform_comparison_sentiment_analysis(df):
    # Create empty lists to store the sentiment scores and labels
    sentiment_scores = []
    sentiment_labels = []

    # Iterate over the 'data' column in the DataFrame
    for text in df['data']:
        # Perform sentiment analysis using the Hugging Face pipeline
        result = sentiment_classifier(text)[0]
        sentiment_score = result['score']
        
        # Determine sentiment label based on sentiment score
        if sentiment_score > 0.6:
            sentiment_label = "positive"
        elif sentiment_score < 0.4:
            sentiment_label = "negative"
        else:
            sentiment_label = "neutral"

        # Append the sentiment score and label to the respective lists
        sentiment_scores.append(sentiment_score)
        sentiment_labels.append(sentiment_label)

    # Add the sentiment scores and labels as new columns in the DataFrame
    df['sentiment_score'] = sentiment_scores
    df['sentiment_label'] = sentiment_labels

    return df


# Perform sentiment analysis on the DataFrame
df = perform_comparison_sentiment_analysis(df_en)

# Print the updated DataFrame
df.head()

'''

'three labels:\ndef perform_comparison_sentiment_analysis(df):\n    # Create empty lists to store the sentiment scores and labels\n    sentiment_scores = []\n    sentiment_labels = []\n\n    # Iterate over the \'data\' column in the DataFrame\n    for text in df[\'data\']:\n        # Perform sentiment analysis using the Hugging Face pipeline\n        result = sentiment_classifier(text)[0]\n        sentiment_score = result[\'score\']\n        \n        # Determine sentiment label based on sentiment score\n        if sentiment_score > 0.6:\n            sentiment_label = "positive"\n        elif sentiment_score < 0.4:\n            sentiment_label = "negative"\n        else:\n            sentiment_label = "neutral"\n\n        # Append the sentiment score and label to the respective lists\n        sentiment_scores.append(sentiment_score)\n        sentiment_labels.append(sentiment_label)\n\n    # Add the sentiment scores and labels as new columns in the DataFrame\n    df[\'sentiment_score\']

In [10]:
print(df['sentiment_label'].unique())
print(df['sentiment_score'].min())
print(df['sentiment_score'].max())

['positiv']
0.5091699957847595
0.6674174070358276


In [11]:
df[df['sentiment_label'] == 'negativ'].shape[0]/df.shape[0]

0.0

In [12]:
# Drop rows where 'Label' is NaN or empty
df_en.dropna(subset=['Label'], inplace=True)

# Print the updated DataFrame
data = df_en[df_en['Label'] != 'neutral']

df_en.head()

,data,player,language,publishedAt,Label,sentiment_score,sentiment_label
1,midfielders leandro paredes juventus angel ...,exequiel palacios,en,2023-03-03T16:42:19Z,neutral,0.650659,positiv
2,half goal joshua kimmich canceled penalties pa...,exequiel palacios,en,2023-03-19T18:30:00Z,positiv,0.612918,positiv
4,bayerleverkusen s palacios scored second half ...,exequiel palacios,en,2023-03-19T19:05:09Z,positiv,0.632688,positiv
6,palacios scored penalties bayerleverkusen beat...,exequiel palacios,en,2023-03-19T20:03:28Z,positiv,0.643328,positiv
8,bayerleverkusen stun bayern munich bayerleverk...,exequiel palacios,en,2023-03-20T04:56:02Z,positiv,0.637220,positiv


In [13]:
data

,data,player,language,publishedAt,Label,sentiment_score,sentiment_label
2,half goal joshua kimmich canceled penalties pa...,exequiel palacios,en,2023-03-19T18:30:00Z,positiv,0.612918,positiv
4,bayerleverkusen s palacios scored second half ...,exequiel palacios,en,2023-03-19T19:05:09Z,positiv,0.632688,positiv
6,palacios scored penalties bayerleverkusen beat...,exequiel palacios,en,2023-03-19T20:03:28Z,positiv,0.643328,positiv
8,bayerleverkusen stun bayern munich bayerleverk...,exequiel palacios,en,2023-03-20T04:56:02Z,positiv,0.637220,positiv
11,foul palacios argentina,exequiel palacios,en,2023-03-29T06:58:57Z,negativ,0.615712,positiv
16,ronald koeman launched scathing assessment fri...,jeremie frimpong,en,2023-03-22T10:35:51Z,negativ,0.521703,positiv
17,ronald koeman undiplomatic return saying over...,jeremie frimpong,en,2023-03-24T07:23:27Z,negativ,0.592686,positiv
19,bayerleverkusen handed schalke defeat matches ...,jeremie frimpong,en,2023-04-01T19:24:43Z,positiv,0.573699,positiv
31,german outfit stormed victory aggregate goals ...,jeremie frimpong,en,2023-04-20T20:52:37Z,positiv,0.606132,positiv
53,diaby wirtz struck break flip game it s head l...,moussa diaby,en,2023-02-16T23:15:39Z,positiv,0.548560,positiv


In [17]:
print(data['sentiment_score'].min())
print(data['sentiment_score'].max())


0.5217034220695496
0.6433281302452087


In [14]:
# Calculate the accuracy
accuracy = (data['sentiment_label'] == data['Label']).mean() * 100

# Print the accuracy
print("Accuracy: {:.2f}%".format(accuracy))

Accuracy: 76.92%


In [15]:
# Assuming you have the true labels in 'Label' column and predicted labels in 'sentiment_label' column of the DataFrame
true_labels = data['Label']
predicted_labels = data['sentiment_label']

# Create the confusion matrix
cm = confusion_matrix(true_labels, predicted_labels)

# Convert the confusion matrix to a DataFrame for better visualization
labels = np.unique(np.concatenate((true_labels, predicted_labels)))
cm_df = pd.DataFrame(cm, index=labels, columns=labels)

# Display the confusion matrix
print("Confusion Matrix:")
print(cm_df)

Confusion Matrix:
         negativ  positiv
negativ        0        3
positiv        0       10


In [16]:
# Assuming you have the true labels in 'Label' column and predicted labels in 'sentiment_label' column of the DataFrame
true_labels = data['Label']
predicted_labels = data['sentiment_label']

# Generate the classification report
report = classification_report(true_labels, predicted_labels)

# Display the classification report
print("Classification Report:")
print(report)


Classification Report:
              precision    recall  f1-score   support

     negativ       0.00      0.00      0.00         3
     positiv       0.77      1.00      0.87        10

    accuracy                           0.77        13
   macro avg       0.38      0.50      0.43        13
weighted avg       0.59      0.77      0.67        13



C:\Users\joana\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\joana\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\joana\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\metrics\_classification.py:1344